In [1]:
from exprel.dataset.semeval_dataset import SemevalDataset

In [2]:
data = SemevalDataset("/home/kovacs/projects/exp-relation-extraction/data/semeval_train.txt")

2021-03-19 15:00:50 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | combined  |
| pos       | combined  |
| lemma     | combined  |
| depparse  | combined  |
| sentiment | sstplus   |
| ner       | ontonotes |

/home/kovacs/miniconda3/envs/rel/lib/python3.9/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 6050). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0
2021-03-19 15:00:50 INFO: Use device: cpu
2021-03-19 15:00:50 INFO: Loading: tokenize
2021-03-19 15:00:50 INFO: Loading: pos
2021-03-19 15:00:50 INFO: Lo

In [7]:
data.to_dataframe().groupby("label_id").size()

label_id
0      344
1      659
2      470
3      471
4      374
5      166
6      844
7        1
8      568
9      148
10      97
11     407
12      78
13     612
14     490
15     144
16    1410
17     323
18     394
dtype: int64